In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#       print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Common Includes

In [3]:
import tensorflow as tf
from tensorflow import keras
print("TF version:", tf.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")
!lscpu | grep 'Model name'
!lscpu | grep 'Socket(s):'
!lscpu | grep 'Core(s) per socket'
!lscpu | grep 'Thread(s) per core'
!nvidia-smi

TF version: 2.6.2
GPU is available


2022-04-14 15:05:52.465601: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-14 15:05:52.544867: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-14 15:05:52.545586: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


Model name:                      Intel(R) Xeon(R) CPU @ 2.00GHz
Socket(s):                       1
Core(s) per socket:              1
Thread(s) per core:              2
Thu Apr 14 15:05:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    25W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |

Dataset

In [8]:
AUTOTUNE = tf.data.AUTOTUNE
BATCH_SIZE = 64

ds_val = tf.keras.utils.image_dataset_from_directory(
    directory = "../input/imagenetmini-1000/imagenet-mini/val",
    labels = "inferred",
    label_mode = "int",
    color_mode = "rgb",
    batch_size = BATCH_SIZE,
    image_size = (224, 224)
)

def normalize_image(image, label):
    image = tf.keras.applications.resnet50.preprocess_input(image)
    #image = tf.cast(x=image, dtype=tf.float32)/127.5 - 1.0
    return image, label

ds_val = ds_val.map(map_func=normalize_image, num_parallel_calls=AUTOTUNE)
ds_val = ds_val.prefetch(buffer_size=AUTOTUNE)

Found 3923 files belonging to 1000 classes.


Model

In [ ]:
INPUT_SHAPE = (224,224,3)

model_full = tf.keras.applications.resnet50.ResNet50(
    include_top=True,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation=None
)


model_full.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(),
    metrics=['accuracy', tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5)]
)

model_full.summary()

Evaluate

In [10]:
model_full.evaluate(ds_val, verbose=1)

62/62 [==============================] - 22s 304ms/step - loss: 1.2508 - accuracy: 0.7030 - sparse_top_k_categorical_accuracy: 0.9011


[1.2508450746536255, 0.7030333876609802, 0.9010961055755615]

Calculate Flops

In [11]:
!pip install keras-flops
from keras_flops import get_flops
flops = get_flops(model_full, batch_size=1)
print(f"FLOPS: {flops / 10 ** 9:.03} G")

     |████████████████████████████████| 14.8 MB 717 kB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=77049 sha256=0b2635925904660ced09d7da43c8ddc35c38578da96fccec9012b74bfc18d73d
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built wrapt
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
  Attempting uninstall: six
    Found existing installation: six 1.16.0
    Uninstalling six-1.16.0:
      Successfully uninstalled six-1.16.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.20.3
    Uninstalling numpy-1.20.3:
      Successfully uninstalled numpy-1.20.3
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.13.3
    Uninstalling wrapt-1.13.3:
 

2022-04-14 15:12:19.807065: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-14 15:12:19.807439: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2022-04-14 15:12:19.807668: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-04-14 15:12:19.808247: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-14 15:12:19.808649: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-14 15:12:19.808993: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] su


=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
scope: The nodes in the model graph are organized by their names, which is hierarchical like filesystem.
flops: Number of float operations. Note: Please read the implementation for the math behind it.

Profi

Save Model

In [12]:
Model_Name = "ResNet50_ImageNet_70_90_7_76GF"
os.environ["MODELNAME"] = Model_Name
os.environ["MODELNAMEZIP"] = Model_Name + ".zip"
model_full.save("./" + Model_Name)
!zip -r ./$MODELNAMEZIP ./$MODELNAME
!rm -rf ./$MODELNAME

2022-04-14 15:13:45.659175: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


  adding: ResNet50_ImageNet_70_90_7_76GF/ (stored 0%)
  adding: ResNet50_ImageNet_70_90_7_76GF/keras_metadata.pb (deflated 95%)
  adding: ResNet50_ImageNet_70_90_7_76GF/variables/ (stored 0%)
  adding: ResNet50_ImageNet_70_90_7_76GF/variables/variables.data-00000-of-00001 (deflated 7%)
  adding: ResNet50_ImageNet_70_90_7_76GF/variables/variables.index (deflated 81%)
  adding: ResNet50_ImageNet_70_90_7_76GF/assets/ (stored 0%)
  adding: ResNet50_ImageNet_70_90_7_76GF/saved_model.pb (deflated 93%)
